In [1]:
import sqlite3
from flask import Flask, render_template, request
import numpy as np
import pandas as pd

In [2]:
#플라스크 객체 생성
app=Flask(__name__)

#-------------------------로그인 페이지-----------------------

#로그인 폼 렌더링
#/ 경로 요청 시 실행 함수
@app.route('/mach1', ) #요청 경로 지정
def login(): #실행 함수 작성
    return render_template('login.html') #login.html 파일 렌더링


#로그인 처리
#/login1 경로 요청 시 실행 함수
@app.route('/login1', methods=['POST']) #요청 경로 지정, form태그의 매개변수 값 전달 방식_POST
def login1(): #실행 함수 작성
    #폼 입력값 가져오기
    idn=request.form['id']
    pwd=request.form['passwd']
    
    #데이터베이스 연결
    conn=sqlite3.connect('member.db')
    conn.row_factory=sqlite3.Row
    cursor=conn.cursor()
    #계정 id 레코드 읽기
    cursor.execute('select*from 회원정보 where id=?',(idn,))
    row=cursor.fetchone()
    conn.close()
    
    #계정/비번 조사
    if row!=None:
        if idn==row['id']and pwd==row['passwd']:
            return render_template('registration.html')
        
    return("<h2>로그인 에러!!</h2>")


In [3]:
#회원가입 폼 렌더링
#/signup 경로 요청 시 실행 함수
@app.route('/signup') #요청 경로 지정
def signup(): #실행 함수 작성
    return render_template('signup.html') #signup.html 파일 렌더링

#회원가입 처리
#/register 경로 요청 시 실행 함수
@app.route('/register',methods=['POST']) ##요청 경로 지정, form태그의 매개변수 값 전달 방식_POST
def register(): #실행 함수 작성
    
    #폼 입력값 가져오기
    idn=request.form['id']
    pwd=request.form['passwd']
    name=request.form['name']
    age=request.form['age']
    act=request.form['act']
    fam=request.form['fam']
    pain=request.form['pain']
    inter=request.form['inter']
    Medu=request.form['Medu']
    Fedu=request.form['Fedu']
    study=request.form['study']
    sch=request.form['sch']

    #데이터베이스 연결
    conn=sqlite3.connect('member.db')
    cursor=conn.cursor()
    #데이터베이스 등록(삽입)
    cursor.execute('''
    insert into 회원정보 (id,passwd,name,age,activities,famsup,pain,internet,Medu,Fedu,studytime,schoolsup) values(?,?,?,?,?,?,?,?,?,?,?,?)''', (idn,pwd,name,age,act,fam,pain,inter,Medu,Fedu,study,sch))
    conn.commit()
    conn.close()
    
    return render_template('login.html') #login.html파일 렌더링


In [4]:
## 선형 회귀분석 모델 ##


# 판다스 데이터 정리
import pandas as pd # 판다스 라이브러리
import csv # csv 라이브러리

tr = pd.read_csv("student.csv", encoding = 'utf-8')

## 필요한 데이터 분리 ##
student_x_data = tr.loc[:, ['age','activities','famsup','paid','internet','Medu','Fedu','studytime','schoolsup','G3']]
student_y_data = tr.loc[:,['pass']]

################################
### 데이터 확인을 위한 명령어###
################################

# tr.head()
# student_x_data.head(40)
# student_y_data.head()


import numpy as np
import matplotlib.pyplot as plt
np.random.seed(1) # 재 실행 시에도 동일한 데이터 생성을 위해 랜덤 시드 값 지정
x_data = student_x_data
y_data = student_y_data


############################
##########파이토치##########
############################

import torch
import torch.nn as nn
import torch.nn.functional as F

# 파이토치 텐서 훈련 데이터 생성
x_train = torch.FloatTensor(x_data.values).unsqueeze(dim=1)
# unsqueeze(dim=1): (1000,) => (1000, 1)

y_train = torch.FloatTensor(y_data.values).unsqueeze(dim=1)

# 모델을 선언 및 초기화. 다중 선형 회귀이므로 input_dim=10, output_dim=1.
model12 = nn.Sequential(
    nn.Linear(10,8), # 입력계층 = 1, 은닉계층1 = 5
    nn.ReLU(), # ReLu 활성화 함수
    nn.Linear(8,3), # 은닉 계층2 = 3
    nn.ReLU(), # ReLu 활성화 함수
    nn.Linear(3,1) # 출력계층 = 1
)
print(list(model12.parameters()))

# optimizer 설정. 경사 하강법 SGD를 사용하고 학습 속도를 의미하는 lr은 0.0
optimizer = torch.optim.SGD(model12.parameters(), lr=0.05)

# 전체 훈련 데이터에 대해 경사 하강법을 100회 반복
nb_epochs=5000

ep_data = []
loss_data = []

for epoch in range(nb_epochs):
    # H(x) 계산
    prediction = model12(x_train)
    
    # 손실함수 계산
    loss = F.mse_loss(prediction, y_train)
    
    # 역전파 수행
    optimizer.zero_grad() # 기울기(gradient)를 0으로 초기화
    loss.backward() # 손실함수를 미분하여 기울기 계산
    optimizer.step() # W와 b를 업데이트
    
    # 10번 반복마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} loss : {:.6}'.format(epoch, nb_epochs, loss.item()))

        ep_data.append([epoch])
        loss_data.append([loss.item()])



print("학습 종료")

c:\Users\mraen\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Parameter containing:
tensor([[-0.0300, -0.1000, -0.2574,  0.0741, -0.1786, -0.0677, -0.0099,  0.2878,
          0.2666,  0.2151],
        [ 0.2177, -0.0360, -0.1504, -0.0759,  0.1924, -0.0128,  0.0165, -0.2253,
         -0.1854, -0.0807],
        [ 0.0037,  0.2157,  0.2758,  0.1835, -0.1698, -0.1430, -0.1451,  0.1566,
          0.2714, -0.2153],
        [-0.1883,  0.2221,  0.3100,  0.3131, -0.1651,  0.0147, -0.2611,  0.0337,
          0.1574,  0.2742],
        [ 0.2494, -0.1450, -0.2996,  0.1201, -0.2977,  0.1302, -0.0804, -0.0600,
         -0.2729, -0.1457],
        [-0.1724,  0.0417, -0.1624, -0.2119,  0.2669,  0.1432, -0.0176,  0.3040,
         -0.3026, -0.1507],
        [-0.2268, -0.0155, -0.2139, -0.1010,  0.0443,  0.0274, -0.0340,  0.1104,
         -0.2249,  0.0960],
        [ 0.2476,  0.1778, -0.2569,  0.1404,  0.1070,  0.1655,  0.0560,  0.2831,
         -0.2743, -0.2473]], requires_grad=True), Parameter containing:
tensor([ 0.1525, -0.0068, -0.1534,  0.2367,  0.2154,  0.1338,

In [5]:
# csv 파일 불러오기
import pandas as pd
import csv

# 파일명
file_name = 'student.csv'
data = pd.read_csv(file_name)

# 데이터 크기 확인 395개의 데이터 29개의 특징, 1개의 라벨데이터
# data.shape
# # 상위 5개 데이터 확인
# data.head(5)

# 필요한 데이터 전처리
X_data = data.loc[:, ['age','activities','famsup','paid','internet','Medu','Fedu','studytime','schoolsup','G3']]
Y_data = data['pass']

# 훈련데이터 크기 확인
# X_data.shape, Y_data.shape

# #X_data, Y_data 확인
# X_data.head(10)
# Y_data.head(10)



import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import TensorDataset, DataLoader

# 토치 플롯 텐서로 변환 - 훈련 데이터
X_train = torch.FloatTensor(X_data.values[10:])
Y_train = torch.FloatTensor(Y_data.values[10:]).long()

# 토치 플롯 텐서로 변환 - 테스트 데이터
X_test = torch.FloatTensor(X_data.values[:10])
Y_test = torch.FloatTensor(Y_data.values[:10]).long()

# 데이터 크기 확인, 훈련데이터 385, 테스트 데이터 10
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape


# 학습용 딥러닝 모델 클래스 생성
class FC_NN(nn.Module):
    def __init__(self):
        super(FC_NN, self).__init__()
        self.input_layer = nn.Linear(10, 5)
        self.hidden_layer = nn.Linear(5, 2)
    def forward(self, x):
        x = self.input_layer(x)
        x = F.relu(x)
        output = self.hidden_layer(x)
        
        return output

# 모델 생성, 모델 옵션 지정    
model = FC_NN()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
epochs = 100

epoch_data = []
loss1_data = []

# 학습 단계
for epoch in range(epochs):
    optimizer.zero_grad()
    prediction = model(X_train)
    loss = loss_fn(prediction, Y_train)
    loss = F.cross_entropy(prediction, Y_train) #
    loss.backward()
    optimizer.step()
    if (epoch) % 10 == 0:
        print(f'Epoch: {epoch}, Loss: {loss : .4f}')

        epoch_data.append([epoch])
        loss1_data.append([loss.item()])





# 학습 검증 단계
import numpy as np

# 테스트 데이터를 넣으면 합격할 확률을 알려준다.
def model_predict(data):
    # 모델에 값 예측
    pred = model(data)
    print(type(data))
    # softmax로 확률로 변경
    for i, j in pred:
        one = np.array([i.item(), j.item()])
        exp_one = np.exp(one)
        sum_exp = np.sum(exp_one)
        output = exp_one / sum_exp
    
    return 100*output[1]

Epoch: 0, Loss:  0.6801
Epoch: 10, Loss:  0.1486
Epoch: 20, Loss:  0.0704
Epoch: 30, Loss:  0.0557
Epoch: 40, Loss:  0.0504
Epoch: 50, Loss:  0.0474
Epoch: 60, Loss:  0.0461
Epoch: 70, Loss:  0.0430
Epoch: 80, Loss:  0.0418
Epoch: 90, Loss:  0.0398


In [6]:
##### 선형회귀 분석 머신러닝 #####
@app.route('/model', methods=['POST'])
def model1():
    score = request.form['score']
    pl_name = request.form['name']
    score_data = []
    conn=sqlite3.connect('member.db')
    conn.row_factory=sqlite3.Row
    cursor = conn.cursor()
    cursor.execute('select*from 회원정보 where name =?',(pl_name,))
    rows=cursor.fetchall()
    conn.close()

    print(pl_name)

    data_list=[int(rows[0]['age']), int(rows[0]['activities']), int(rows[0]['famsup']),int(rows[0]['pain']),int(rows[0]['internet']),int(rows[0]['Medu']),int(rows[0]['Fedu']),int(rows[0]['studytime']),int(rows[0]['schoolsup']),int(score)]

    print(data_list)

    
    new_var = torch.FloatTensor([data_list])
    pred_y = model12(new_var)
    XX_DA = model_predict(new_var)
    print(f"라벨 데이터가 9인 학생의 합격할 확률은 {model_predict(new_var) : .2f} % 입니다.")

    return render_template('model.html', play_name = pl_name, play_data1 = pred_y*100, play_data2 = XX_DA, map1=map)
   

In [7]:
@app.route("/loss1", methods=['POST'])
def loss1():

    import base64
    from io import BytesIO
    from matplotlib.figure import Figure
    
    fig = Figure()
    ax = fig.subplots()
    ax.plot(ep_data, loss_data)
    buf = BytesIO()
    fig.savefig(buf, format="png")
    data = base64.b64encode(buf.getbuffer()).decode("ascii")

    return f"<img src='data:image/png;base64,{data}'/>"

In [8]:
@app.route("/loss2", methods=['POST'])
def loss2():

    import base64
    from io import BytesIO
    from matplotlib.figure import Figure
    
    fig = Figure()
    ax = fig.subplots()
    ax.plot(epoch_data, loss1_data)
    buf = BytesIO()
    fig.savefig(buf, format="png")
    data = base64.b64encode(buf.getbuffer()).decode("ascii")

    return f"<img src='data:image/png;base64,{data}'/>"

In [9]:
# 간단한 신경망 모델 loss 값 확인
print(loss1_data)
print(epoch_data)

# 선형 회귀분석 모델 loss 값 확인
print("\n")
print(ep_data)
print(loss_data)

[[0.6800834536552429], [0.1485721319913864], [0.07040438801050186], [0.055710919201374054], [0.05037609487771988], [0.047393158078193665], [0.04611106961965561], [0.04301177337765694], [0.04175527021288872], [0.039751820266246796]]
[[0], [10], [20], [30], [40], [50], [60], [70], [80], [90]]


[[0], [100], [200], [300], [400], [500], [600], [700], [800], [900], [1000], [1100], [1200], [1300], [1400], [1500], [1600], [1700], [1800], [1900], [2000], [2100], [2200], [2300], [2400], [2500], [2600], [2700], [2800], [2900], [3000], [3100], [3200], [3300], [3400], [3500], [3600], [3700], [3800], [3900], [4000], [4100], [4200], [4300], [4400], [4500], [4600], [4700], [4800], [4900]]
[[1.0306307077407837], [0.09778676927089691], [0.08566739410161972], [0.08035551011562347], [0.06334741413593292], [0.0680479183793068], [0.05945270508527756], [0.053117070347070694], [0.04685777425765991], [0.041757989674806595], [0.035986632108688354], [0.033456284552812576], [0.03013783134520054], [0.027444690465

In [10]:
#플라스크 서버 구동
if __name__=='__main__':
    app.run()  #실행되는 컴퓨터 IP 주소, 포트번호 5000
    

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [13/Dec/2022 02:39:08] "GET /melon HTTP/1.1" 404 -
127.0.0.1 - - [13/Dec/2022 02:39:17] "GET /ㅣㅐ햐ㅜ HTTP/1.1" 404 -
